In [1]:
import polars as pl
from enhance_parquet import enhance_option_aggs_parquet, enhance_stock_aggs_parquet, enhance_treasury_parquet

In [2]:
option_df = pl.read_parquet("test_data/current_options_day_aggs.parquet")
option_df = enhance_option_aggs_parquet(option_df)
option_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,6]"
"""O:A250815C00095000""",1,23.78,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",95.000000
"""O:A250815C00100000""",1,19.00,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",100.000000
"""O:A250815C00110000""",7,8.50,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",110.000000
"""O:A250815C00115000""",31,3.75,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",115.000000
"""O:A250815C00120000""",181,0.65,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",120.000000


In [3]:
stock_df = pl.read_parquet("test_data/current_stocks_day_aggs.parquet")
stock_df = enhance_stock_aggs_parquet(stock_df)
stock_df.head()

ticker,volume,price,timestamp
str,i64,"decimal[*,2]",datetime[ns]
"""A""",926367,119.23,2025-08-14 04:00:00
"""AA""",4725672,31.08,2025-08-14 04:00:00
"""AAA""",28251,25.02,2025-08-14 04:00:00
"""AAAA""",123447,25.85,2025-08-14 04:00:00
"""AAAU""",1606776,33.01,2025-08-14 04:00:00


In [4]:
treasury_df = pl.read_parquet("test_data/treasury_yields.parquet")
treasury_df = enhance_treasury_parquet(treasury_df)

treasury_df = treasury_df.select('date', '3m')
current_date = pl.datetime(2025, 8, 14)
treasury_df = treasury_df.filter(
    pl.col('date') == current_date
)
treasury_df = treasury_df.select('date', '3m')
treasury_df = treasury_df.rename({'date': 'treasury_date', '3m': 'risk_free_rate'})
treasury_df = treasury_df.with_columns(
    pl.col('treasury_date').dt.date().alias('treasury_date')
)

treasury_df.tail()

treasury_date,risk_free_rate
date,f64
2025-08-14,0.043


In [5]:
# Combine everything into one df for calculating IV and Greeks

# Add current underlying price to options df
stock_df = stock_df.select('ticker', 'price')
stock_df = stock_df.rename({'ticker': 'underlying_ticker'})
combined_df = option_df.join(stock_df, on='underlying_ticker')

# Get the risk free rate using the 3m yield from the treasury df
combined_df = combined_df.with_columns(
    pl.col('timestamp').dt.date().alias('treasury_date')
)
combined_df = combined_df.join(treasury_df, on='treasury_date')
combined_df = combined_df.drop('treasury_date')

combined_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price,price,risk_free_rate
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,6]","decimal[*,2]",f64
"""O:A250815C00095000""",1,23.78,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",95.000000,119.23,0.043
"""O:A250815C00100000""",1,19.00,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",100.000000,119.23,0.043
"""O:A250815C00110000""",7,8.50,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",110.000000,119.23,0.043
"""O:A250815C00115000""",31,3.75,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",115.000000,119.23,0.043
"""O:A250815C00120000""",181,0.65,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",120.000000,119.23,0.043


In [6]:
# Going to need time to expirations for IV and Greek calculations

combined_df = combined_df.with_columns([
    ((pl.col('expiration_date').dt.date() - pl.col('timestamp').dt.date()).dt.total_days() / 365.25)
    .alias('time_to_expiry_years'),
    (pl.col('expiration_date').dt.date() - pl.col('timestamp').dt.date()).dt.total_days()
    .alias('time_to_expiry_days')
])

combined_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price,price,risk_free_rate,time_to_expiry_years,time_to_expiry_days
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,6]","decimal[*,2]",f64,f64,i64
"""O:A250815C00095000""",1,23.78,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",95.000000,119.23,0.043,0.002738,1
"""O:A250815C00100000""",1,19.00,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",100.000000,119.23,0.043,0.002738,1
"""O:A250815C00110000""",7,8.50,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",110.000000,119.23,0.043,0.002738,1
"""O:A250815C00115000""",31,3.75,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",115.000000,119.23,0.043,0.002738,1
"""O:A250815C00120000""",181,0.65,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",120.000000,119.23,0.043,0.002738,1


In [7]:
# Methods for calculating IV

import numpy as np

def calculate_implied_vol_vectorized(batch):
    """
    Vectorized implied volatility calculation using the working pandas formula
    Converted to work with Polars map_batches
    """
    market_prices, S_values, K_values, T_values, r_values, option_types = batch
    
    # Convert to numpy arrays
    S = np.array([float(x) for x in S_values])      # asset_price
    K = np.array([float(x) for x in K_values])      # strike_price  
    T = np.array([max(float(x), 1/365.25) for x in T_values])  # time to expiration in years
    C = np.array([float(x) for x in market_prices]) # option_price
    r = float(r_values[0])  # risk_free_rate (same for all in batch)
    
    intrinsic = np.clip(K - S * np.exp(-r * T), a_min=0, a_max=None)
    numerator = C + intrinsic / 2
    iv_approx = np.sqrt(2 * np.pi / T) * (numerator / S)
    
    result = []
    for v in iv_approx:
        if 0.01 < v < 2.0:
            result.append(v)
        else:
            result.append(0.0)  # Default for out-of-bounds values
    
    return pl.Series(result, dtype=pl.Float64)

In [8]:
# Add IV to each option in the df

combined_df = combined_df.with_columns([
    pl.map_batches(
        exprs=[
            pl.col('option_price').cast(pl.Float64),
            pl.col('price').cast(pl.Float64), 
            pl.col('strike_price').cast(pl.Float64),
            pl.col('time_to_expiry_years'),
            pl.col('risk_free_rate'),
            pl.col('option_type')
        ],
        function=calculate_implied_vol_vectorized,
        return_dtype=pl.self_dtype()
    ).alias('implied_volatility')
])

combined_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price,price,risk_free_rate,time_to_expiry_years,time_to_expiry_days,implied_volatility
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,6]","decimal[*,2]",f64,f64,i64,f64
"""O:A250815C00095000""",1,23.78,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",95.000000,119.23,0.043,0.002738,1,0.0
"""O:A250815C00100000""",1,19.00,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",100.000000,119.23,0.043,0.002738,1,0.0
"""O:A250815C00110000""",7,8.50,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",110.000000,119.23,0.043,0.002738,1,0.0
"""O:A250815C00115000""",31,3.75,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",115.000000,119.23,0.043,0.002738,1,1.506714
"""O:A250815C00120000""",181,0.65,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",120.000000,119.23,0.043,0.002738,1,0.418673


In [9]:
# Methods for calculating an options greeks

def simple_option_price(S, K, T, r, sigma, option_type):
    """Simple option pricing using finite difference approximation"""
    if T <= 0:
        if option_type == 'CALL':
            return max(S - K, 0)
        else:
            return max(K - S, 0)
    
    # Much simpler approximation that's more stable
    # Based on basic option pricing principles
    
    # Moneyness
    if option_type == 'CALL':
        moneyness = S / K
        intrinsic = max(S - K, 0)
    else:
        moneyness = K / S  
        intrinsic = max(K - S, 0)
    
    # Time value approximation
    time_value = sigma * S * np.sqrt(T) * 0.4
    
    # Adjust for moneyness
    if moneyness > 1.05:  # ITM
        time_value *= 0.5  # Less time value for ITM
    elif moneyness < 0.95:  # OTM
        time_value *= 0.3  # Even less time value for OTM
    
    return intrinsic + time_value

def calculate_greeks_finite_diff(S, K, T, r, sigma, option_type):
    """Calculate Greeks using finite difference methods with corrected theta"""
    
    # Small perturbations for finite differences
    dS = S * 0.01  # 1% move in stock price
    dT = 1/365.25  # 1 day time decay
    dSigma = 0.01  # 1% volatility change
    dr = 0.0001    # 1 basis point rate change
    
    # Base option price
    base_price = simple_option_price(S, K, T, r, sigma, option_type)
    
    # Delta: dPrice/dS
    price_up = simple_option_price(S + dS, K, T, r, sigma, option_type)
    price_down = simple_option_price(S - dS, K, T, r, sigma, option_type)
    delta = (price_up - price_down) / (2 * dS)
    
    # Gamma: d²Price/dS²
    gamma = (price_up - 2 * base_price + price_down) / (dS * dS)
    
    # Theta: Use a simpler, more accurate approach
    # For short-term options, theta ≈ -time_value / days_remaining
    if option_type == 'CALL':
        intrinsic = max(S - K, 0)
    else:
        intrinsic = max(K - S, 0)
    
    time_value = max(base_price - intrinsic, 0)
    days_remaining = T * 365.25
    
    if days_remaining > 1:
        # Simple theta: lose some fraction of time value per day
        theta = -(time_value * 0.1) / days_remaining  # Lose ~10% of time value per day for short-term
    else:
        theta = -time_value  # Lose all remaining time value on last day
    
    # Clamp theta to reasonable bounds
    theta = max(-1.0, min(0.0, theta))
    
    # Vega: dPrice/dSigma
    price_vol_up = simple_option_price(S, K, T, r, sigma + dSigma, option_type)
    vega = (price_vol_up - base_price) / dSigma / 100  # Per 1% vol change
    
    # Rho: dPrice/dr  
    price_rate_up = simple_option_price(S, K, T, r + dr, sigma, option_type)
    rho = (price_rate_up - base_price) / dr / 100  # Per 1% rate change
    
    return delta, gamma, theta, vega, rho

def calculate_greeks_vectorized(batch):
    """Vectorized Greeks calculation using finite differences"""
    S_values, K_values, T_values, r_values, sigma_values, option_types = batch
    
    # Create lists to store all Greeks as nested lists
    all_greeks = []
    
    for i in range(len(S_values)):
        try:
            delta, gamma, theta, vega, rho = calculate_greeks_finite_diff(
                float(S_values[i]),      # S
                float(K_values[i]),      # K
                max(float(T_values[i]), 1/365.25),  # T
                float(r_values[i]),      # r
                float(sigma_values[i]),  # sigma
                option_types[i]          # option_type
            )
            all_greeks.append([delta, gamma, theta, vega, rho])
        except Exception as e:
            # Handle any errors with default values
            all_greeks.append([0.0, 0.0, -0.01, 0.0, 0.0])
    
    # Return as a Series of lists
    return pl.Series(all_greeks)

In [10]:
# Add greeks to each options row

combined_df = combined_df.with_columns([
    pl.map_batches(
        exprs=[
            pl.col('price').cast(pl.Float64),
            pl.col('strike_price').cast(pl.Float64),
            pl.col('time_to_expiry_years'),
            pl.col('risk_free_rate'),
            pl.col('implied_volatility'),
            pl.col('option_type')
        ],
        function=calculate_greeks_vectorized,
        return_dtype=pl.List(pl.Float64)
    ).alias('greeks_results')
]).with_columns([
    pl.col('greeks_results').list.get(0).alias('delta'),
    pl.col('greeks_results').list.get(1).alias('gamma'),
    pl.col('greeks_results').list.get(2).alias('theta'),
    pl.col('greeks_results').list.get(3).alias('vega'),
    pl.col('greeks_results').list.get(4).alias('rho')
]).drop([
    'time_to_expiry_years', 'greeks_results'
])

combined_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price,price,risk_free_rate,time_to_expiry_days,implied_volatility,delta,gamma,theta,vega,rho
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,6]","decimal[*,2]",f64,i64,f64,f64,f64,f64,f64,f64
"""O:A250815C00095000""",1,23.78,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",95.000000,119.23,0.043,1,0.0,1.0,0.0,0.0,0.012477,0.0
"""O:A250815C00100000""",1,19.00,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",100.000000,119.23,0.043,1,0.0,1.0,0.0,0.0,0.012477,0.0
"""O:A250815C00110000""",7,8.50,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",110.000000,119.23,0.043,1,0.0,1.0,0.0,0.0,0.012477,0.0
"""O:A250815C00115000""",31,3.75,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",115.000000,119.23,0.043,1,1.506714,1.031535,-1.2496e-15,-1.0,0.024955,0.0
"""O:A250815C00120000""",181,0.65,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",120.000000,119.23,0.043,1,0.418673,0.185857,0.297064,-1.0,0.024955,0.0


In [14]:
# Test it out

spy_df = combined_df.filter(pl.col('underlying_ticker') == "SPY").filter(pl.col('time_to_expiry_days') == 0).filter(pl.col('strike_price') == 640.00)

spy_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price,price,risk_free_rate,time_to_expiry_days,implied_volatility,delta,gamma,theta,vega,rho
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,6]","decimal[*,2]",f64,i64,f64,f64,f64,f64,f64,f64
"""O:SPY250814C00640000""",13698,3.36,2025-08-14 04:00:00,"""SPY""",2025-08-14 00:00:00,"""CALL""",640.000000,642.79,0.043,0,0.250412,0.722264,0.088047,-1.0,0.134535,0.0
"""O:SPY250814P00640000""",177834,0.60,2025-08-14 04:00:00,"""SPY""",2025-08-14 00:00:00,"""PUT""",640.000000,642.79,0.043,0,0.044716,-0.282041,0.088047,-0.601591,0.134535,0.0


In [ ]:
# Putting it all together 

def calculate_implied_vol_vectorized(batch):
    """
    Vectorized implied volatility calculation using the working pandas formula
    Converted to work with Polars map_batches
    """
    market_prices, S_values, K_values, T_values, r_values, option_types = batch
    
    # Convert to numpy arrays
    S = np.array([float(x) for x in S_values])      # asset_price
    K = np.array([float(x) for x in K_values])      # strike_price  
    T = np.array([max(float(x), 1/365.25) for x in T_values])  # time to expiration in years
    C = np.array([float(x) for x in market_prices]) # option_price
    r = float(r_values[0])  # risk_free_rate (same for all in batch)
    
    intrinsic = np.clip(K - S * np.exp(-r * T), a_min=0, a_max=None)
    numerator = C + intrinsic / 2
    iv_approx = np.sqrt(2 * np.pi / T) * (numerator / S)
    
    result = []
    for v in iv_approx:
        if 0.01 < v < 2.0:
            result.append(v)
        else:
            result.append(0.0)  # Default for out-of-bounds values
    
    return pl.Series(result, dtype=pl.Float64)


def simple_option_price(S, K, T, r, sigma, option_type):
    """Simple option pricing using finite difference approximation"""
    if T <= 0:
        if option_type == 'CALL':
            return max(S - K, 0)
        else:
            return max(K - S, 0)
    
    # Moneyness
    if option_type == 'CALL':
        moneyness = S / K
        intrinsic = max(S - K, 0)
    else:
        moneyness = K / S  
        intrinsic = max(K - S, 0)
    
    # Time value approximation
    time_value = sigma * S * np.sqrt(T) * 0.4
    
    # Adjust for moneyness
    if moneyness > 1.05:  # ITM
        time_value *= 0.5  # Less time value for ITM
    elif moneyness < 0.95:  # OTM
        time_value *= 0.3  # Even less time value for OTM
    
    return intrinsic + time_value


def calculate_greeks_finite_diff(S, K, T, r, sigma, option_type):
    """Calculate Greeks using finite difference methods with corrected theta"""
    
    # Small perturbations for finite differences
    dS = S * 0.01  # 1% move in stock price
    dT = 1/365.25  # 1 day time decay
    dSigma = 0.01  # 1% volatility change
    dr = 0.0001    # 1 basis point rate change
    
    # Base option price
    base_price = simple_option_price(S, K, T, r, sigma, option_type)
    
    # Delta: dPrice/dS
    price_up = simple_option_price(S + dS, K, T, r, sigma, option_type)
    price_down = simple_option_price(S - dS, K, T, r, sigma, option_type)
    delta = (price_up - price_down) / (2 * dS)
    
    # Gamma: d²Price/dS²
    gamma = (price_up - 2 * base_price + price_down) / (dS * dS)
    
    # Theta: Use a simpler, more accurate approach
    # For short-term options, theta ≈ -time_value / days_remaining
    if option_type == 'CALL':
        intrinsic = max(S - K, 0)
    else:
        intrinsic = max(K - S, 0)
    
    time_value = max(base_price - intrinsic, 0)
    days_remaining = T * 365.25
    
    if days_remaining > 1:
        # Simple theta: lose some fraction of time value per day
        theta = -(time_value * 0.1) / days_remaining  # Lose ~10% of time value per day for short-term
    else:
        theta = -time_value  # Lose all remaining time value on last day
    
    # Clamp theta to reasonable bounds
    theta = max(-1.0, min(0.0, theta))
    
    # Vega: dPrice/dSigma
    price_vol_up = simple_option_price(S, K, T, r, sigma + dSigma, option_type)
    vega = (price_vol_up - base_price) / dSigma / 100  # Per 1% vol change
    
    # Rho: dPrice/dr  
    price_rate_up = simple_option_price(S, K, T, r + dr, sigma, option_type)
    rho = (price_rate_up - base_price) / dr / 100  # Per 1% rate change
    
    return delta, gamma, theta, vega, rho


def calculate_greeks_vectorized(batch):
    """Vectorized Greeks calculation using finite differences"""
    S_values, K_values, T_values, r_values, sigma_values, option_types = batch
    
    # Create lists to store all Greeks as nested lists
    all_greeks = []
    
    for i in range(len(S_values)):
        try:
            delta, gamma, theta, vega, rho = calculate_greeks_finite_diff(
                float(S_values[i]),      # S
                float(K_values[i]),      # K
                max(float(T_values[i]), 1/365.25),  # T
                float(r_values[i]),      # r
                float(sigma_values[i]),  # sigma
                option_types[i]          # option_type
            )
            all_greeks.append([delta, gamma, theta, vega, rho])
        except Exception as e:
            # Handle any errors with default values
            all_greeks.append([0.0, 0.0, -0.01, 0.0, 0.0])
    
    # Return as a Series of lists
    return pl.Series(all_greeks)


option_df = pl.read_parquet("test_data/current_options_day_aggs.parquet")
option_df = enhance_option_aggs_parquet(option_df)

stock_df = pl.read_parquet("test_data/current_stocks_day_aggs.parquet")
stock_df = enhance_stock_aggs_parquet(stock_df)

# Add current underlying price to options df
stock_df = stock_df.select('ticker', 'price')
stock_df = stock_df.rename({'ticker': 'underlying_ticker'})
combined_df = option_df.join(stock_df, on='underlying_ticker')

# Calculate DTE for greek calculation
combined_df = combined_df.with_columns([
    ((pl.col('expiration_date').dt.date() - pl.col('timestamp').dt.date()).dt.total_days() / 365.25)
    .alias('time_to_expiry_years'),
    (pl.col('expiration_date').dt.date() - pl.col('timestamp').dt.date()).dt.total_days()
    .alias('time_to_expiry_days')
])

# Filter treasury down to relevant date with 3m yield for risk free rate
treasury_df = pl.read_parquet("test_data/treasury_yields.parquet")
treasury_df = enhance_treasury_parquet(treasury_df)
treasury_df = treasury_df.select('date', '3m')
current_date = pl.datetime(2025, 8, 14)
treasury_df = treasury_df.filter(
    pl.col('date') == current_date
)
treasury_df = treasury_df.select('date', '3m')
treasury_df = treasury_df.rename({'date': 'treasury_date', '3m': 'risk_free_rate'})
treasury_df = treasury_df.with_columns(
    pl.col('treasury_date').dt.date().alias('treasury_date')
)

# Add risk free rate column
combined_df = combined_df.with_columns(
    pl.col('timestamp').dt.date().alias('treasury_date')
)
combined_df = combined_df.join(treasury_df, on='treasury_date')
combined_df = combined_df.drop('treasury_date')

# Add IV to each option in the df
combined_df = combined_df.with_columns([
    pl.map_batches(
        exprs=[
            pl.col('option_price').cast(pl.Float64),
            pl.col('price').cast(pl.Float64), 
            pl.col('strike_price').cast(pl.Float64),
            pl.col('time_to_expiry_years'),
            pl.col('risk_free_rate'),
            pl.col('option_type')
        ],
        function=calculate_implied_vol_vectorized,
        return_dtype=pl.self_dtype()
    ).alias('implied_volatility')
])

# Add Greeks to each option
combined_df = combined_df.with_columns([
    pl.map_batches(
        exprs=[
            pl.col('price').cast(pl.Float64),
            pl.col('strike_price').cast(pl.Float64),
            pl.col('time_to_expiry_years'),
            pl.col('risk_free_rate'),
            pl.col('implied_volatility'),
            pl.col('option_type')
        ],
        function=calculate_greeks_vectorized,
        return_dtype=pl.List(pl.Float64)
    ).alias('greeks_results')
]).with_columns([
    pl.col('greeks_results').list.get(0).alias('delta'),
    pl.col('greeks_results').list.get(1).alias('gamma'),
    pl.col('greeks_results').list.get(2).alias('theta'),
    pl.col('greeks_results').list.get(3).alias('vega'),
    pl.col('greeks_results').list.get(4).alias('rho')
]).drop([
    'time_to_expiry_years', 'greeks_results'
])




In [2]:
%pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [QuantLib-Python]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# american_iv_only_api.py
from dataclasses import dataclass
from typing import Dict, Any, Optional
import polars as pl
import QuantLib as ql

def _qdate(iso: str) -> ql.Date:
    y, m, d = map(int, iso.split("-"))
    return ql.Date(d, m, y)

@dataclass
class AmericanCfg:
    use_binomial: bool = True
    binomial_steps: int = 2500
    fd_t_grid: int = 200
    fd_x_grid: int = 250
    day_count: ql.DayCounter = ql.Actual365Fixed()
    calendar: ql.Calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
    iv_guess: float = 0.25         # starting guess for solver
    iv_min: float = 1e-6
    iv_max: float = 5.0
    vega_bump: float = 1e-4        # for numerical vega

class AmericanIVGreeks:
    def __init__(self, cfg: AmericanCfg = AmericanCfg()):
        self.cfg = cfg
        self.eval_date = ql.Date.todaysDate()
        ql.Settings.instance().evaluationDate = self.eval_date

    def _process(self, spot: float, r: float, q: float, vol: float) -> ql.BlackScholesMertonProcess:
        rf = ql.YieldTermStructureHandle(ql.FlatForward(self.eval_date, r, self.cfg.day_count))
        dv = ql.YieldTermStructureHandle(ql.FlatForward(self.eval_date, q, self.cfg.day_count))
        vt = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(self.eval_date, self.cfg.calendar, vol, self.cfg.day_count))
        return ql.BlackScholesMertonProcess(ql.QuoteHandle(ql.SimpleQuote(spot)), dv, rf, vt)

    def _engine(self, process):
        if self.cfg.use_binomial:
            return ql.BinomialVanillaEngine(process, "crr", self.cfg.binomial_steps)
        return ql.FdBlackScholesVanillaEngine(
            process, tGrid=self.cfg.fd_t_grid, xGrid=self.cfg.fd_x_grid
        )

    def _price_greeks(self, opt, process) -> Dict[str, float]:
        eng = self._engine(process); opt.setPricingEngine(eng)
        return {"price": opt.NPV(), "delta": opt.delta(), "gamma": opt.gamma(), "theta": opt.theta()}

    def _vega_num(self, opt, process) -> float:
        base_vol = process.blackVolatility()
        sigma0 = base_vol.blackVol(0.0, 1.0)
        bumped = ql.BlackConstantVol(self.eval_date, self.cfg.calendar, sigma0 + self.cfg.vega_bump, self.cfg.day_count)
        base_vol.linkTo(bumped); opt.setPricingEngine(self._engine(process))
        up = opt.NPV()
        base_vol.linkTo(ql.BlackConstantVol(self.eval_date, self.cfg.calendar, sigma0, self.cfg.day_count))
        opt.setPricingEngine(self._engine(process))
        return (up - opt.NPV()) / self.cfg.vega_bump

    def _american_iv(self, opt, process, market_price: float) -> float:
        # QuantLib uses the provided process as context; vol is iterated internally
        try:
            return opt.impliedVolatility(market_price, process,
                                         1e-6, 800, self.cfg.iv_min, self.cfg.iv_max)
        except Exception:
            return float("nan")

    def row(self, s: Dict[str, Any]) -> Dict[str, Any]:
        spot = float(s["spot"]); strike = float(s["strike"]); is_call = bool(s["is_call"])
        r = float(s.get("r", 0.0)); q = float(s.get("q", 0.0))
        maturity = _qdate(str(s["maturity"]))
        market_px = float(s["market_price"])

        payoff = ql.PlainVanillaPayoff(ql.Option.Call if is_call else ql.Option.Put, strike)
        exercise = ql.AmericanExercise(self.eval_date, maturity)
        opt = ql.VanillaOption(payoff, exercise)

        # 1) Solve IV using a process with a guess vol
        proc_guess = self._process(spot, r, q, self.cfg.iv_guess)
        iv = self._american_iv(opt, proc_guess, market_px)

        # 2) Re-price at IV and compute Greeks
        if iv == iv:  # not NaN
            proc_iv = self._process(spot, r, q, iv)
            out = self._price_greeks(opt, proc_iv)
            vega = self._vega_num(opt, proc_iv)
            return {
                "iv": iv,
                "price_iv": out["price"],
                "delta_iv": out["delta"],
                "gamma_iv": out["gamma"],
                "theta_iv": out["theta"] / 365,
                "vega_iv": vega,
            }
        else:
            return {
                "iv": None, "price_iv": None, "delta_iv": None,
                "gamma_iv": None, "theta_iv": None, "vega_iv": None
            }

def add_iv_and_greeks(df: pl.DataFrame,
                      pricer: Optional[AmericanIVGreeks] = None) -> pl.DataFrame:
    """
    Requires columns: spot, strike, is_call, maturity (YYYY-MM-DD), r, q, market_price
    Adds: iv, price_iv, delta_iv, gamma_iv, theta_iv, vega_iv
    """
    need = ["spot","strike","is_call","maturity","r","q","market_price"]
    miss = [c for c in need if c not in df.columns]
    if miss:
        raise ValueError(f"Missing required columns: {miss}")
    pricer = pricer or AmericanIVGreeks()
    tmp = df.with_columns(pl.struct(need).map_elements(pricer.row).alias("_m"))
    return tmp.with_columns(
        pl.col("_m").struct.field("iv").alias("iv"),
        pl.col("_m").struct.field("price_iv").alias("price_iv"),
        pl.col("_m").struct.field("delta_iv").alias("delta_iv"),
        pl.col("_m").struct.field("gamma_iv").alias("gamma_iv"),
        pl.col("_m").struct.field("theta_iv").alias("theta_iv"),
        pl.col("_m").struct.field("vega_iv").alias("vega_iv"),
    ).drop("_m")
    
    


In [ ]:
import polars as pl
from enhance_parquet import CleanParquetUtility

df = pl.DataFrame({
    "spot":[100,100,100],
    "strike":[95,105,110],
    "is_call":[True,False,False],
    "maturity":["2026-01-16","2026-01-16","2026-06-19"],
    "r":[0.03,0.03,0.03],
    "q":[0.01,0.01,0.01],
    "market_price":[7.80,8.75,11.2],
})

cfg = AmericanCfg(use_binomial=True, binomial_steps=2500)
pricer = AmericanIVGreeks(cfg)

option_df = pl.read_parquet("test_data/current_options_day_aggs.parquet")
option_df = CleanParquetUtility.option_aggs_parquet(option_df)

stock_df = pl.read_parquet("test_data/current_stocks_day_aggs.parquet")
stock_df = CleanParquetUtility.stock_aggs_parquet(stock_df)

treasury_df = pl.read_parquet("test_data/treasury_yields.parquet")
treasury_df = CleanParquetUtility.treasury_parquet(treasury_df)

# Add current underlying price to options df
stock_df = stock_df.select('ticker', 'price')
stock_df = stock_df.rename({'ticker': 'underlying_ticker', 'price': 'spot'})
combined_df = option_df.join(stock_df, on='underlying_ticker')

# Get risk free rate
treasury_df = treasury_df.select('date', '3m')
current_date = pl.datetime(2025, 8, 14)
treasury_df = treasury_df.filter(
    pl.col('date') == current_date
)
treasury_df = treasury_df.select('date', '3m')
treasury_df = treasury_df.rename({'date': 'treasury_date', '3m': 'risk_free_rate'})
treasury_df = treasury_df.with_columns(
    pl.col('treasury_date').dt.date().alias('treasury_date')
)

# Add risk free rate to options column
combined_df = combined_df.with_columns(
    pl.col('timestamp').dt.date().alias('treasury_date')
)
combined_df = combined_df.join(treasury_df, on='treasury_date')
combined_df = combined_df.drop('treasury_date')

#Rename columns for compatibility
combined_df = combined_df.rename({'strike_price': 'strike', 'risk_free_rate': 'r'})

#res = add_iv_and_greeks(df, pricer=pricer)
#print(res)

combined_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price,spot,risk_free_rate
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,6]","decimal[*,2]",f64
"""O:A250815C00095000""",1,23.78,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",95.000000,119.23,0.043
"""O:A250815C00100000""",1,19.00,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",100.000000,119.23,0.043
"""O:A250815C00110000""",7,8.50,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",110.000000,119.23,0.043
"""O:A250815C00115000""",31,3.75,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",115.000000,119.23,0.043
"""O:A250815C00120000""",181,0.65,2025-08-14 04:00:00,"""A""",2025-08-15 00:00:00,"""CALL""",120.000000,119.23,0.043
